In [1]:
%config Completer.use_jedi = False

# Load wine data

In [2]:
import pandas as pd

In [3]:
wine = pd.read_csv('https://bit.ly/wine_csv_data')

In [4]:
wine.head()

,alcohol,sugar,pH,class
0,9.4,1.9,3.51,0.0
1,9.8,2.6,3.20,0.0
2,9.8,2.3,3.26,0.0
3,9.8,1.9,3.16,0.0
4,9.4,1.9,3.51,0.0


In [5]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

print(data.shape)
print(target.shape)

(6497, 3)
(6497,)


# Split data

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
train_input, test_input, train_target, test_target = train_test_split(data,
                                                                      target,
                                                                      test_size=0.2,
                                                                      random_state=42)

In [8]:
sub_input, val_input, sub_target, val_target = train_test_split(train_input,
                                                                train_target,
                                                                test_size=0.2,
                                                                random_state=42)

In [10]:
print(sub_input.shape)
print(val_input.shape)

(4157, 3)
(1040, 3)


# Build a decision tree model

In [11]:
from sklearn.tree import DecisionTreeClassifier

In [12]:
dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)

DecisionTreeClassifier(random_state=42)

In [13]:
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


# Cross validation

In [14]:
from sklearn.model_selection import cross_validate

### 5-fold

In [15]:
scores = cross_validate(dt, train_input, train_target)
scores

{'fit_time': array([0.00986004, 0.00840187, 0.00819397, 0.00802827, 0.00791097]),
 'score_time': array([0.00094891, 0.00086927, 0.00090027, 0.00077105, 0.00076699]),
 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}

In [16]:
import numpy as np

In [17]:
np.mean(scores['test_score'])

0.855300214703487

### 10-fold

In [18]:
from sklearn.model_selection import StratifiedKFold

In [19]:
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
np.mean(scores['test_score'])

0.8574181117533719

# Grid search

In [20]:
from sklearn.model_selection import GridSearchCV

In [21]:
params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)

In [22]:
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [23]:
dt = gs.best_estimator_
dt.score(train_input, train_target)

0.9615162593804117

In [24]:
gs.best_params_

{'min_impurity_decrease': 0.0001}

In [25]:
gs.cv_results_['mean_test_score']

array([0.86819297, 0.86453617, 0.86492226, 0.86780891, 0.86761605])

In [27]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])
gs.cv_results_['params'][best_index]

{'min_impurity_decrease': 0.0001}

In [29]:
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),
          'max_depth': range(5, 20, 1),
          'min_samples_split': range(2, 100, 10)}

In [30]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [31]:
gs.best_params_

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}

In [32]:
np.max(gs.cv_results_['mean_test_score'])

0.8683865773302731

# Random search

In [33]:
from scipy.stats import uniform, randint

In [35]:
rgen = randint(0, 10)
rgen.rvs(10)

array([4, 9, 9, 5, 9, 8, 4, 1, 7, 1])

In [36]:
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([ 95, 112,  95,  98,  94, 107,  99, 103,  93, 104]))

In [37]:
ugen = uniform(0, 1)
ugen.rvs(10)

array([0.77963334, 0.63102028, 0.6946001 , 0.03972428, 0.74567874,
       0.36192377, 0.54893429, 0.90342744, 0.626634  , 0.63442857])

In [38]:
params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth': randint(20, 50),
          'min_samples_split': randint(2, 25),
          'min_samples_leaf': randint(1, 25)}

# Random search

In [39]:
from sklearn.model_selection import RandomizedSearchCV

In [40]:
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42),
                        params,
                        n_iter=100,
                        n_jobs=-1,
                        random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x120c14190>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_frozen object at 0x120c14a90>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x120c14160>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x120d99400>},
                   random_state=42)

In [41]:
gs.best_params_

{'max_depth': 39,
 'min_impurity_decrease': 0.00034102546602601173,
 'min_samples_leaf': 7,
 'min_samples_split': 13}

In [42]:
np.max(gs.cv_results_['mean_test_score'])

0.8695428296438884

In [ ]:
dt = gs.best_estimator_
dt.score(test_input, test_target)

# Application problem

In [44]:
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42, splitter='random'),
                        params,
                        n_iter=100,
                        n_jobs=-1,
                        random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42,
                                                    splitter='random'),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x120c14190>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_frozen object at 0x120c14a90>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x120c14160>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x120d99400>},
                   random_state=42)

In [45]:
gs.best_params_

{'max_depth': 43,
 'min_impurity_decrease': 0.00011407982271508446,
 'min_samples_leaf': 19,
 'min_samples_split': 18}

In [46]:
dt = gs.best_estimator_

In [47]:
dt.score(train_input, train_target)

0.8043101789493938

In [48]:
dt.score(test_input, test_target)

0.786923076923077